In [ ]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from arch import arch_model
import yfinance as yf
from tqdm import tqdm

In [ ]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database['Close'][i+dif])-np.log(Database['Close'][i]))
    return np.append(np.repeat(np.nan, dif),Out), Database.index

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))


#Database is calculated
def DatabaseGeneration (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns,'TrueSD': TrueSD})
    Data = Data.set_index(Index) 
    return Data.dropna()

In [ ]:
start='2008-01-01'; end='2015-12-31'; 
asset = "^GSPC"
Lag=1; LagSD=4
IndexEndDays=yf.download(asset,start=start,  end=end, progress=False).resample('W-FRI').last().index
Database=yf.download(asset,start, end, progress=False).resample('W-FRI').last()
Data = DatabaseGeneration(Database, Lag, LagSD)
Data

In [ ]:
#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off')
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=4).variance.dropna())
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11

In [ ]:

# asset_name = re.sub('[\W\d_]+', '', asset)
IndexEndDays=yf.download(asset,start=start,  end=end, progress=False).resample('W-FRI').last().index

Lag=1; LagSD=5; Timestep=10; Dropout=0; LearningRate=0.01; Epochs=100

DataValidation = DatabaseGeneration(yf.download(asset,start='2000-01-01', end=date.today()+timedelta(days=1), progress=False).resample('W-FRI').last(), Lag, LagSD)

ResultsCollection=pd.DataFrame({'Date_Forecast': [], 'h1': [], 'h2': [], 'h3':[], 'h4': []})
#Loop for generating the results
for i in tqdm(range(IndexEndDays.shape[0])):
    #Database is downloaded from yahoo finance and lag of returns defined
    Database=yf.download(asset,start=IndexEndDays[i].date()-timedelta(days=780), end=IndexEndDays[i].date(), progress=False).resample('W-FRI').last()
    #Database for fitting the models is generated
    Data = DatabaseGeneration(Database, Lag, LagSD)
    #Fitting of Transformed ANN-ARCH model, ARCH models and forecasting of the next volatility value

    ARCH11, res_GARCH11, CV_GARCH11, GARCH = GARCH_Model_Student(Data)
    Date_Forcast = IndexEndDays[i]

    IterResults={'Date_Forecast':Date_Forcast.date(),'h1': GARCH[0][0]/100,'h2': GARCH[0][1]/100,'h3': GARCH[0][2]/100,'h4': GARCH[0][3]/100,'TrueSD':DataValidation[DataValidation.index==Date_Forcast]['TrueSD'][0]}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    ResultsCollection = ResultsCollection.append(IterResults_df, ignore_index=True)

    ResultsCollection.to_csv(f'./Evaluation/GCH_1.csv',index=False)
